Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import sys
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [ ]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

In [ ]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

text = read_data(filename)
print('Data size %d' % len(text))

Create a small validation set.

In [ ]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

Utility functions to map characters to vocabulary IDs and back.

In [ ]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0

def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Function to generate a training batch for the LSTM model.

In [ ]:
class BatchGenerator(object):
    def __init__(self, text, size, unrolls):
        self._text = text
        self._text_size = len(text)
        self._batch_size = size
        self._unrolls = unrolls
        segment = self._text_size // size
        self._cursor = [ offset * segment for offset in range(size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by unrolls new ones.
        """
        batches = [self._last_batch]
        for step in range(self._unrolls):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, 5, 10)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(train_text[:30])

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

In [ ]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def make_one_hot(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_single_output():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [ ]:
def construct_verbose(node_count, batch_size, unrolls):
    graph = tf.Graph()
    with graph.as_default():
        # Parameters:
        # Input gate: input, previous output, and bias.
        ix = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        im = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        ib = tf.Variable(tf.zeros([1, node_count]))
        # Forget gate: input, previous output, and bias.
        fx = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        fm = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        fb = tf.Variable(tf.zeros([1, node_count]))
        # Memory cell: input, state and bias.                             
        cx = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        cm = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        cb = tf.Variable(tf.zeros([1, node_count]))
        # Output gate: input, previous output, and bias.
        ox = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        om = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        ob = tf.Variable(tf.zeros([1, node_count]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([node_count, vocabulary_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([vocabulary_size]))

        # Definition of the cell computation.
        def lstm_cell(i, o, state):
            """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
            Note that in this formulation, we omit the various connections between the
            previous state and the gates."""
            input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
            forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
            update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
            state = forget_gate * state + input_gate * tf.tanh(update)
            output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
            return output_gate * tf.tanh(state), state

        # Input data.
        train_data = list()
        for _ in range(unrolls + 1):
            train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
        train_inputs = train_data[:unrolls]
        train_labels = train_data[1:]  # labels are inputs shifted by one time step.

        # Unrolled LSTM loop.
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs:
            output, state = lstm_cell(i, output, state)
            outputs.append(output)

        # State saving across unrollings.
        with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
            # Classifier.
            logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        # Predictions.
        train_prediction = tf.nn.softmax(logits)

        # Sampling and validation eval: batch 1, no unrolling.
        sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
        saved_sample_output = tf.Variable(tf.zeros([1, node_count]))
        saved_sample_state = tf.Variable(tf.zeros([1, node_count]))
        reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, node_count])),
            saved_sample_state.assign(tf.zeros([1, node_count]))
        )
        sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
        with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                      saved_sample_state.assign(sample_state)]):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    return {
        "graph": graph,
        "batch_size": batch_size,
        "unrolls": unrolls,
        "train_data": train_data,
        "optimizer": optimizer,
        "loss": loss,
        "train_prediction": train_prediction,
        "learning_rate": learning_rate,
        "reset_sample_state": reset_sample_state,
        "sample_input": sample_input,
        "sample_prediction": sample_prediction,
        "output_to_input": make_one_hot,
        "random_output": random_single_output
    }

In [ ]:
def run_lstm(setup, batcher, training, validation, step_count, report_every):
    train_batches = batcher(training, setup["batch_size"], setup["unrolls"])
    valid_batches = batcher(validation, 1, 1)
    with tf.Session(graph=setup["graph"]) as session:
        tf.initialize_all_variables().run()
        print('Initialized')
        mean_loss = 0
        for step in range(step_count + 1):
            batches = train_batches.next()
            feed_dict = dict()
            for i in range(setup["unrolls"] + 1):
                feed_dict[setup["train_data"][i]] = batches[i]

            _, l, predictions, lr = session.run(
                [setup["optimizer"], setup["loss"], setup["train_prediction"], setup["learning_rate"]],
                feed_dict=feed_dict
            )

            mean_loss += l
            if step % report_every == 0:
                if step > 0:
                    mean_loss = mean_loss / report_every
                # The mean loss is an estimate of the loss over the last few batches.
                print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
                mean_loss = 0
                labels = np.concatenate(list(batches)[1:])
                print(predictions.shape)
                print(labels)
                print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
                if step % (report_every * 10) == 0:
                    # Generate some samples.
                    print('=' * 80)
                    for _ in range(5):
                        feed = setup["output_to_input"](setup["random_output"]())
                        sentence = characters(feed)[0]
                        setup["reset_sample_state"].run()
                        for _ in range(79):
                            prediction = setup["sample_prediction"].eval({setup["sample_input"]: feed})
                            feed = setup["output_to_input"](prediction)
                            sentence += characters(feed)[0]
                        print(sentence)
                    print('=' * 80)
                # Measure validation set perplexity.
                setup["reset_sample_state"].run()
                valid_logprob = 0
                for _ in range(len(validation)):
                    b = valid_batches.next()
                    predictions = setup["sample_prediction"].eval({setup["sample_input"]: b[0]})
                    valid_logprob = valid_logprob + logprob(predictions, b[1])
                print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

In [ ]:
verbose_setup = construct_verbose(node_count=64, batch_size=64, unrolls=10)

run_lstm(verbose_setup, BatchGenerator, train_text, valid_text, step_count=7000, report_every=100)

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [ ]:
def construct_compact(node_count, batch_size, unrolls):
    graph = tf.Graph()
    with graph.as_default():
        gate_count = 4
        # Parameters:
        # Gates: input, previous output, and bias.
        input_weights = tf.Variable(tf.truncated_normal([vocabulary_size, node_count * gate_count], -0.1, 0.1))
        output_weights = tf.Variable(tf.truncated_normal([node_count, node_count * gate_count], -0.1, 0.1))
        bias = tf.Variable(tf.zeros([1, node_count * gate_count]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([node_count, vocabulary_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([vocabulary_size]))

        # Definition of the cell computation.
        def lstm_cell(i, o, state):
            """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
            Note that in this formulation, we omit the various connections between the
            previous state and the gates."""
            values = tf.split(1, gate_count, tf.matmul(i, input_weights) + tf.matmul(o, output_weights) + bias)
            input_gate = tf.sigmoid(values[0])
            forget_gate = tf.sigmoid(values[1])
            update = values[2]
            state = forget_gate * state + input_gate * tf.tanh(update)
            output_gate = tf.sigmoid(values[3])
            return output_gate * tf.tanh(state), state

        # Input data.
        train_data = list()
        for _ in range(unrolls + 1):
            train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
        train_inputs = train_data[:unrolls]
        train_labels = train_data[1:]  # labels are inputs shifted by one time step.

        # Unrolled LSTM loop.
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs:
            output, state = lstm_cell(i, output, state)
            outputs.append(output)

        # State saving across unrollings.
        with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
            # Classifier.
            logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        # Predictions.
        train_prediction = tf.nn.softmax(logits)

        # Sampling and validation eval: batch 1, no unrolling.
        sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
        saved_sample_output = tf.Variable(tf.zeros([1, node_count]))
        saved_sample_state = tf.Variable(tf.zeros([1, node_count]))
        reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, node_count])),
            saved_sample_state.assign(tf.zeros([1, node_count]))
        )
        sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
        with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                      saved_sample_state.assign(sample_state)]):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    return {
        "graph": graph,
        "batch_size": batch_size,
        "unrolls": unrolls,
        "train_data": train_data,
        "optimizer": optimizer,
        "loss": loss,
        "train_prediction": train_prediction,
        "learning_rate": learning_rate,
        "reset_sample_state": reset_sample_state,
        "sample_input": sample_input,
        "sample_prediction": sample_prediction,
        "output_to_input": make_one_hot,
        "random_output": random_single_output
    }

In [ ]:
compact_setup = construct_compact(node_count=64, batch_size=64, unrolls=10)

run_lstm(compact_setup, BatchGenerator, train_text, valid_text, step_count=7000, report_every=100)

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [ ]:
unique_bigrams = vocabulary_size * vocabulary_size

def encode_bigram(characters):
    first = char2id(characters[0])
    second = (char2id(characters[1]) if len(characters) > 1 else 0)
    return first * vocabulary_size + second

def decode_bigram(bigram_id):
    second = bigram_id % vocabulary_size
    first = (bigram_id - second) // vocabulary_size
    return "".join([id2char(first), id2char(second)])

class BigramBatchGenerator(object):
    def __init__(self, text, size, unrolls):
        self._text = text
        self._text_size = len(text)
        self._batch_size = size
        self._unrolls = unrolls
        segment = self._text_size // (2 * size)
        self._cursor = [offset * segment for offset in range(size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size), dtype=np.int)
        for b in range(self._batch_size):
            batch[b] = encode_bigram(self._text[self._cursor[b]:self._cursor[b] + 2])
            self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._unrolls):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def bigrams(encoded_bigrams):
    """Turn encoded bigrams back into its (most likely) character representation."""
    return [decode_bigram(int(b)) for b in encoded_bigrams]

def bibatches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, bigrams(b))]
    return s

In [ ]:
print(train_text[:30])

bigram_batches = BigramBatchGenerator(train_text, 5, 10)

print(bibatches2string(bigram_batches.next()))
print(bibatches2string(bigram_batches.next()))
print(bibatches2string(bigram_batches.next()))
print(bibatches2string(bigram_batches.next()))

In [ ]:
def construct_bigram(node_count, batch_size, embedding_size, unrolls):
    graph = tf.Graph()
    with graph.as_default():
        gate_count = 4
        # Parameters:
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size * vocabulary_size, embedding_size], -1.0, 1.0))
        
        # Gates: input, previous output, and bias.
        input_weights = tf.Variable(tf.truncated_normal([embedding_size, node_count * gate_count], -0.1, 0.1))
        output_weights = tf.Variable(tf.truncated_normal([node_count, node_count * gate_count], -0.1, 0.1))
        bias = tf.Variable(tf.zeros([1, node_count * gate_count]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([node_count, embedding_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([embedding_size]))

        # Definition of the cell computation.
        def lstm_cell(i, o, state):
            """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
            Note that in this formulation, we omit the various connections between the
            previous state and the gates."""
            values = tf.split(1, gate_count, tf.matmul(i, input_weights) + tf.matmul(o, output_weights) + bias)
            input_gate = tf.sigmoid(values[0])
            forget_gate = tf.sigmoid(values[1])
            update = values[2]
            state = forget_gate * state + input_gate * tf.tanh(update)
            output_gate = tf.sigmoid(values[3])
            return output_gate * tf.tanh(state), state

        # Input data.
        train_data = list()
        train_embeddings = list()
        for _ in range(unrolls + 1):
            train_data.append(tf.placeholder(tf.int32, shape=[batch_size]))
            train_embeddings.append(tf.nn.embedding_lookup(embeddings, train_data[-1]))
        train_inputs = train_embeddings[:unrolls]
        train_labels = train_embeddings[1:]  # labels are inputs shifted by one time step.

        # Unrolled LSTM loop.
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs:
            output, state = lstm_cell(i, output, state)
            outputs.append(output)

        # State saving across unrollings.
        with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
            # Classifier.
            predicted_embeddings = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        # Predictions.
        train_prediction = tf.nn.softmax(logits)

        embedding_norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        embedding_normalized = embeddings / embedding_norm

        # Sampling and validation eval: batch 1, no unrolling.
        sample_input = tf.placeholder(tf.int32, shape=[1])
        saved_sample_output = tf.Variable(tf.zeros([1, node_count]))
        saved_sample_state = tf.Variable(tf.zeros([1, node_count]))
        reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, node_count])),
            saved_sample_state.assign(tf.zeros([1, node_count]))
        )
        sample_embedding = tf.nn.embedding_lookup(embeddings, sample_input)
        sample_output, sample_state = lstm_cell(sample_embedding, saved_sample_output, saved_sample_state)
        with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                      saved_sample_state.assign(sample_state)]):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    return {
        "graph": graph,
        "batch_size": batch_size,
        "unrolls": unrolls,
        "train_data": train_data,
        "optimizer": optimizer,
        "loss": loss,
        "train_prediction": train_prediction,
        "learning_rate": learning_rate,
        "reset_sample_state": reset_sample_state,
        "sample_input": sample_input,
        "sample_prediction": sample_prediction,
        "output_to_input": lambda x: x,
        "random_output": lambda: random.randint(0, vocabulary_size * vocabulary_size - 1)
    }

In [ ]:
bigram_setup = construct_bigram(node_count=64, batch_size=64, embedding_size=8, unrolls=5)

run_lstm(bigram_setup, BigramBatchGenerator, train_text, valid_text, step_count=7000, report_every=100)

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---